<a href="https://colab.research.google.com/github/JSJeong-me/RPA-2023/blob/main/%EC%98%88%EC%B8%A1%EB%B6%84%EC%84%9D/10_model_0520.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_excel("lotto1068.xlsx")

In [3]:
df.head()

,회차,번호1,번호2,번호3,번호4,번호5,번호6
0,1,10,23,29,33,37,40
1,2,9,13,21,25,32,42
2,3,11,16,19,21,27,31
3,4,14,27,30,31,40,42
4,5,16,24,29,40,41,42


In [4]:
df.tail()

,회차,번호1,번호2,번호3,번호4,번호5,번호6
1062,1063,3,6,22,23,24,38
1063,1064,3,6,9,18,22,35
1064,1065,6,11,16,18,21,32
1065,1066,7,10,19,23,28,33
1066,1067,0,0,0,0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   회차      1067 non-null   int64
 1   번호1     1067 non-null   int64
 2   번호2     1067 non-null   int64
 3   번호3     1067 non-null   int64
 4   번호4     1067 non-null   int64
 5   번호5     1067 non-null   int64
 6   번호6     1067 non-null   int64
dtypes: int64(7)
memory usage: 58.5 KB


In [6]:
df.drop(['회차'], axis=1, inplace=True)

In [ ]:
df.describe()

In [ ]:
df.head()

In [7]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [8]:
transformed_df.head()

,0,1,2,3,4,5
0,0.618462,1.438842,1.209283,0.887420,0.606640,0.098565
1,0.431363,-0.011280,0.139157,-0.161606,-0.113846,0.466288
2,0.805561,0.423756,-0.128375,-0.686119,-0.834333,-1.556188
3,1.366859,2.018891,1.343049,0.625163,1.038932,0.466288
4,1.741057,1.583854,1.209283,1.805317,1.183029,0.466288


In [9]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

1067

In [10]:
# Amount of games we need to take into consideration for prediction
window_length = 7
window_length 

7

In [11]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

6

In [ ]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
X

In [ ]:
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
y

In [ ]:
X.shape

In [ ]:
y.shape

In [14]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [15]:
X.shape

(1060, 7, 6)

In [16]:
y.shape

(1060, 6)

In [ ]:
X[0]

In [ ]:
X[1]

In [ ]:
y[0]

In [ ]:
y[1]

In [21]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 2000

In [ ]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(59))
# Adding the last output layer
model.add(Dense(number_of_features))

In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [ ]:
model.fit(x=X, y=y, batch_size=100, epochs=10000, verbose=2)

In [22]:
from tensorflow import keras
model = keras.models.load_model("model_1067_05_10_10000.h5")

In [23]:
to_predict = df.tail(8)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-23-4b4c855b4438>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,번호1,번호2,번호3,번호4,번호5,번호6
1059,3,10,24,33,38,45
1060,4,24,27,35,37,45
1061,20,31,32,40,41,45
1062,3,6,22,23,24,38
1063,3,6,9,18,22,35
1064,6,11,16,18,21,32
1065,7,10,19,23,28,33


In [24]:
to_predict = np.array(to_predict)
to_predict

array([[ 3, 10, 24, 33, 38, 45],
       [ 4, 24, 27, 35, 37, 45],
       [20, 31, 32, 40, 41, 45],
       [ 3,  6, 22, 23, 24, 38],
       [ 3,  6,  9, 18, 22, 35],
       [ 6, 11, 16, 18, 21, 32],
       [ 7, 10, 19, 23, 28, 33]])

In [25]:
scaled_to_predict = scaler.transform(to_predict)

In [26]:
y_pred = model.predict(np.array([scaled_to_predict]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 3s 3s/step
The predicted numbers in the last lottery game are: [ 5 12 18 25 31 34]


In [27]:
prediction = df.tail(1)
prediction = np.array(prediction)
print("The actual numbers in the last lottery game were:", prediction[0])

The actual numbers in the last lottery game were: [0 0 0 0 0 0]


### Save 모델

In [ ]:
model.save("model_1067_05_10_10000.h5")